In [1]:
import re
import nltk
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

Hinkley Location

In [2]:
#https://www.octoparse.com/blog/web-scraping-using-python
urls = ['https://www.yelp.com/biz/grand-casino-hinckley',
      'https://www.yelp.com/biz/grand-casino-hinckley?start=20',
      'https://www.yelp.com/biz/grand-casino-hinckley?start=40']

clean_reviews_hinckley = []
clean_stars_hinckley = []

for url in urls:
    review = []
    stars = []
    ourUrl = urllib.request.urlopen(url)
    soup = BeautifulSoup(ourUrl,'html.parser')

    for i in soup.find_all("li"):
        per_review = i.find_all("div",{"class":"review-content"})
        for z in per_review:
            text = z.find("p")
            review.append(text)
            
    for i in soup.find_all('img', alt = True):
        if "star" in str(i):
            stars.append(i['alt'].replace(' star rating',''))
            
    for each in review:
        new_each = str(each).replace('<br/>','').replace('\xa0','')
        new_each=new_each[13:-4]
        clean_reviews_hinckley.append(new_each)
    
    clean_stars_hinckley.append(stars[1:-2])
    
stars_hinckley = [item for sublist in clean_stars_hinckley for item in sublist]


stars_hinckley.pop(51)
stars_hinckley.pop(38)
stars_hinckley.pop(37)
stars_hinckley.pop(36)
stars_hinckley.pop(13)
stars_hinckley.pop(6)
stars_hinckley.pop(5)

'1.0'

In [3]:
len(clean_reviews_hinckley)

60

In [4]:
len(stars_hinckley)

60

Onamia Location

In [5]:
#https://www.octoparse.com/blog/web-scraping-using-python
urls = ['https://www.yelp.com/biz/grand-casino-onamia-2',
      'https://www.yelp.com/biz/grand-casino-onamia-2?start=20',
      'https://www.yelp.com/biz/grand-casino-onamia-2?start=40']

clean_reviews_onamia = []
clean_stars_onamia = []

for url in urls:
    review = []
    stars = []
    ourUrl = urllib.request.urlopen(url)
    soup = BeautifulSoup(ourUrl,'html.parser')

    for i in soup.find_all("li"):
        per_review = i.find_all("div",{"class":"review-content"})
        for z in per_review:
            text = z.find("p")
            review.append(text)
            
    for i in soup.find_all('img', alt = True):
        if "star" in str(i):
            stars.append(i['alt'].replace(' star rating',''))
            
    for each in review:
        new_each = str(each).replace('<br/>','').replace('\xa0','')
        new_each=new_each[13:-4]
        clean_reviews_onamia.append(new_each)
    
    clean_stars_onamia.append(stars[1:-2])
    
stars_onamia = [item for sublist in clean_stars_onamia for item in sublist]
stars_onamia.pop(7) #this is an updated one

'1.0'

In [6]:
len(clean_reviews_onamia)

50

In [7]:
len(stars_onamia)

50

##### Now Combine

In [114]:
data_hinckley = list(zip(clean_reviews_hinckley, binary_stars_hinckley))
data_onamia = list(zip(clean_reviews_onamia, binary_stars_onamia))

In [115]:
binary_stars_onamia = []
for x in stars_onamia_int:
    if x >= 4:
        binary_stars_onamia.append(1)
    else:
        binary_stars_onamia.append(0)
        
binary_stars_hinckley = []
for x in stars_hinckley_int:
    if x >= 4:
        binary_stars_hinckley.append(1)
    else:
        binary_stars_hinckley.append(0)

Combine the two files

In [116]:
data_hinckley = list(zip(clean_reviews_hinckley, binary_stars_hinckley))
data_onamia = list(zip(clean_reviews_onamia, binary_stars_onamia))

data = data_hinckley + data_onamia

Bootstrap the data to get meaningful weights from the linear model

In [119]:
from sklearn.utils import resample
import numpy as np

boot = resample(data, replace=True, n_samples=1000, random_state=1989)
boot = np.asarray(boot)

features = boot[:,0]
labels = boot[:,1]

Process the text features to normalize

In [1]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

NameError: name 'features' is not defined

In [123]:
import nltk
nltk.download('punkt')
from nltk.text import Text
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ameyers1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
no matches


In [124]:
#ha and wa show up too often. Let's correct these. Ahead of lemmatizing
import re
for x in range(0,len(processed_features)):
    processed_features[x] = re.sub(r'\bha\b', 'has', processed_features[x])
    processed_features[x] = re.sub(r'\bwa\b', 'was', processed_features[x])

In [125]:
nltk.download('wordnet')

def get_lemmatized_text(corpus):
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) 
            for review in corpus]

processed_features = get_lemmatized_text(processed_features)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ameyers1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [126]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ameyers1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [127]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.util import ngrams

vectorizer = TfidfVectorizer(max_features=1000, min_df=7, max_df=0.8, 
                             ngram_range=(3,3), analyzer = 'word', 
                             stop_words=stopwords.words('english'))
processed_features_vec = vectorizer.fit_transform(processed_features).toarray()

Split the features and labels into training and testing files

In [128]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    processed_features_vec, labels, test_size=0.2, random_state=0)

Train the linear model

In [129]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=500, tol=1e-3)
clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=500, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=0.001, verbose=0, warm_start=False)

In [130]:
predictions = clf.predict(X_test)

Evaluate Model

In [167]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(roc_auc_score(y_test.astype(int), predictions.astype(int)))
print(accuracy_score(y_test, predictions))

[[147   0]
 [ 30  23]]
             precision    recall  f1-score   support

          0       0.83      1.00      0.91       147
          1       1.00      0.43      0.61        53

avg / total       0.88      0.85      0.83       200

0.7169811320754718
0.85


Pull output

In [132]:
output = pd.DataFrame(list(zip(vectorizer.get_feature_names(),
                               list(clf.coef_.flatten()))))

output = output.sort_values(by=[1])

In [137]:
output[output[1]<=-2]

,0,1
842,think grand casino,-8.430190
927,wa good hotel,-6.354808
791,smoky nursing home,-5.281178
93,black jack dealer,-4.546143
525,live music event,-4.546143
832,sure lung cancer,-4.331710
146,casino grand casino,-4.146251
25,able get room,-4.070226
775,slot video poker,-4.057301
624,penny nickel slot,-4.017469


Write output to file!

In [138]:
output[output[1]<=-2].to_csv(r"C:\Users\ameyers1\Desktop\bad_trigrams.csv")